# How do I _copy_ a file from another one of _my projects_?
## Overview
Files are stored on the Platform in two spaces: [Public Reference Files](http://docs.sevenbridges.com/docs/file-repositories) and projects of which you're a member. Files can be copied from these two repositories.

## Objective
Here, we focus on **copying a file from your own project**. [[reference](http://docs.sevenbridges.com/docs/copy-a-file)]

## Procedure
1. We'll list all our projects to find our source and target projects.
2. Then, we'll list the files within the two projects.
3. Lastly, we'll copy a file from the source project to the target project.

## Prerequisites
 1. You need to be a member (or owner) of at least one project. Learn more about creating a new project. [[recipe](projects_makeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)].
 2. You need your **authentication token** and the API needs to know about it. See <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a> for details. Learn more about [obtaining your authentication token](http://docs.sevenbridges.com/v1.0/docs/get-your-authentication-token).
 3. You understand how to list projects of which you are a member. We will use this call directly to select a project below. [[recipe](projects_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-your-projects)]
 5. You understand how to list files within a project. We will use this call below and select a file. [[recipe](files_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-files-primary-method)]

Note that as with all okAPI recipes, this recipe makes use of the [Seven Bridges Public API Python library](../Tutorials/SBPLAT/Setup_API_environment.ipynb)
 
## Imports
We import the `Api` class from the official `sevenbridges-python` bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy a file from another project
First, we will list all projects to find our source project, `source\_project`, and our target project, `my\_project_`. Then, we'll list the files within the source project. We'll check that this file doesn't already exist in the target project. If it doesn't exist, we'll copy the file from `source\_project` to `my\_project`.

The critical information for this POST is the **`file_id`**. Note, you are allow to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your `MY\_PROJECT` (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the `name` in the `data` dictionary to something like 'Dept of Awesome.copy'.

In [ ]:
# [USER INPUT] Set your project name; source project name; and file (f_) indices here:
source_project_name = 'Keep on Smiling'        
my_project_name = 'cicipici'
f_index = 0                                     # file to copy


# Find TARGET and YOUR projects
source_project = [p for p in api.projects.query(limit=100).all()
                  if p.name == source_project_name]

my_project = [p for p in api.projects.query(limit=100).all() 
                  if p.name == my_project_name]

# Double-check that all projects exist
if not source_project:
    print('Source project {} not found, check spelling'.format(
        source_project_name))
    raise KeyboardInterrupt
else:
    source_project = source_project[0]
    
if not my_project:
    print('Target project {} not found, check spelling'.format(
            my_project_name))
    raise KeyboardInterrupt
else:
    my_project = my_project[0]

# LIST all files in the source and target project
my_files = api.files.query(limit = 100, project = my_project)
source_files = api.files.query(limit = 100, project = source_project)

# pop out the file names 
my_file_names = [f.name for f in my_files.all()]
source_file_names = [f.name for f in source_files.all()]

# Check if first file already exists in the target project
if source_file_names[f_index] in my_file_names:
    print('File already exists in second project, skipping.')
else:
    print('File {} does not exist in project {}; copying now.'.format(
            source_file_names[f_index], my_project.id))
    
    source_file = [f for f in source_files.all() 
                  if f.name == source_file_names[f_index]][0]
    
    my_new_file = source_file.copy(project = my_project.id,
                                   name = source_file.name)

    # re-list files in target project to verify the copy worked
    my_files = [f.name for f in api.files.query(
            limit = 100, project = my_project.id).all()]
    
    if source_file.name in my_files:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-a-file).